In [1]:
import pandas as pd

# Dataset semanal con demanda intermitente
data = {
    'itemcode': ['SKU_001'] * 20,
    'ds': pd.date_range(start='2024-01-01', periods=20, freq='W'),
    'cantidad': [0, 0, 4, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 7, 0, 0, 2, 0, 0, 6],
    'mesaño': ['2024-' + str(d.isocalendar().week).zfill(2) for d in pd.date_range(start='2024-01-01', periods=20, freq='W')]
}

df_sku = pd.DataFrame(data)


In [2]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import CrostonOptimized



def train_croston_statsforecast(df, freq='W', alpha=None):
    """
    Entrena Croston clásico con statsforecast para un solo itemcode,
    divide en train/test, predice y retorna test con columna Prediccion.

    - df: DataFrame con columnas ['itemcode', 'ds', 'cantidad', 'mesaño']
    - Retorna: df_test con misma estructura + columna Prediccion
    """
    # **Corrección 1:** Añadir una aserción para verificar que solo hay un itemcode
    assert df['itemcode'].nunique() == 1, "El DataFrame debe contener un solo itemcode"
    
    # Asegurar formatos
    # **Corrección 2:** Usar .copy() para evitar SettingWithCopyWarning
    df_copy = df.copy()
    df_copy['ds'] = pd.to_datetime(df_copy['ds'])
    df_copy = df_copy.sort_values('ds').reset_index(drop=True)

    # Separar en train/test
    split_idx = int(len(df_copy) * 0.8)
    df_train = df_copy.iloc[:split_idx].copy()
    df_test = df_copy.iloc[split_idx:].copy()
    h = len(df_test)

    # Preparar datos para el modelo (cambiando nombres de columnas)
    df_train_model = df_train.rename(columns={'itemcode': 'unique_id', 'cantidad': 'y'})[['unique_id', 'ds', 'y']]

    model = CrostonOptimized()
    sf = StatsForecast(models=[model], freq=freq, n_jobs=1)
    
    # 2. Entrenar el modelo final con todos los datos de entrenamiento
    sf.fit(df=df_train_model)

    # 3. Predecir los próximos h pasos
    forecast = sf.predict(h=h)

    # Agregar la columna Prediccion al dataframe de prueba original.
    df_test['Prediccion'] = forecast['CrostonOptimized'].values

    return df_test



d:\maestria umsa\Ciencia de datos introduccion\2. Clase\2. Workshop con Pandas\Datos\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Llamar a la función con el DataFrame de ejemplo
test_result = train_croston_statsforecast(df_sku)
print("----------------- Resultado de la predicción -----------------")
print(test_result)

----------------- Resultado de la predicción -----------------
   itemcode         ds  cantidad   mesaño  Prediccion
16  SKU_001 2024-04-28         2  2024-17    1.222513
17  SKU_001 2024-05-05         0  2024-18    1.222513
18  SKU_001 2024-05-12         0  2024-19    1.222513
19  SKU_001 2024-05-19         6  2024-20    1.222513


In [4]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import CrostonSBA # Importamos el modelo específico para SBA

def train_sba_statsforecast(df, freq='W'):
   
    df_copy = df.copy()
 
    # 3. Separar en conjuntos de entrenamiento y prueba
    split_idx = int(len(df_copy) * 0.8)
    df_train = df_copy.iloc[:split_idx].copy()
    df_test = df_copy.iloc[split_idx:].copy()
    h = len(df_test)

    # 4. Preparar el DataFrame de entrenamiento para StatsForecast
    df_train_model = df_train.rename(
        columns={'itemcode': 'unique_id', 'cantidad': 'y'}
    )[['unique_id', 'ds', 'y']]

    
    model = CrostonSBA()
        
    model_name = 'CrostonSBA'
    
    sf = StatsForecast(models=[model],freq=freq)
        
        # Entrenar el modelo final con todos los datos de entrenamiento
    sf.fit(df=df_train_model)

        # Predecir los próximos 'h' pasos
    forecast = sf.predict(h=h)

        # Agregar la columna de predicción al DataFrame de prueba
    df_test['Prediccion'] = forecast[model_name].values

    return df_test

In [5]:
# Llamar a la función con el DataFrame de ejemplo
test_result = train_sba_statsforecast(df_sku)
print("----------------- Resultado de la predicción -----------------")
print(test_result)

----------------- Resultado de la predicción -----------------
   itemcode         ds  cantidad   mesaño  Prediccion
16  SKU_001 2024-04-28         2  2024-17    1.277884
17  SKU_001 2024-05-05         0  2024-18    1.277884
18  SKU_001 2024-05-12         0  2024-19    1.277884
19  SKU_001 2024-05-19         6  2024-20    1.277884


In [6]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import TSB

# Implementación de MASE (Error Porcentual Absoluto Escalonado)
def mase(y_true, y_pred, y_train):
    """
    Calcula el Error Porcentual Absoluto Escalonado (MASE).
    
    Args:
        y_true (np.array): Valores reales.
        y_pred (np.array): Valores pronosticados.
        y_train (np.array): Valores del conjunto de entrenamiento.
        
    Returns:
        float: El valor del MASE.
    """
    # Numerador: Error absoluto de las predicciones
    abs_errors = np.abs(y_true - y_pred)
    
    # Denominador: Error absoluto del pronóstico ingenuo (naive forecast)
    # y_train[1:] - y_train[:-1] calcula las diferencias de un paso
    denominator = np.mean(np.abs(y_train[1:] - y_train[:-1]))
    
    # Evita la división por cero si el denominador es 0
    if denominator == 0:
        return np.inf # O un valor muy grande para indicar un error
    
    return np.mean(abs_errors) / denominator

def train_tsb_statsforecast_tuned(df, freq='W'):
    """
    Entrena el modelo TSB con ajuste de parámetros (alpha_d y alpha_p) usando validación cruzada,
    optimizando el MASE.
    
    Args:
        df (pd.DataFrame): DataFrame de entrada con columnas ['itemcode', 'ds', 'cantidad'].
        freq (str): Frecuencia de la serie temporal. Ejemplo: 'D', 'W', 'M'.
        
    Returns:
        pd.DataFrame: El DataFrame de prueba con la columna 'Prediccion',
                      'Mejor_MASE' y 'Mejores_Parametros'.
    """
    # Asegurar formato correcto
    df_copy = df.copy()
    df_copy['ds'] = pd.to_datetime(df_copy['ds'])
    df_copy = df_copy.sort_values('ds').reset_index(drop=True)

    # Separar train/test
    split_idx = int(len(df_copy) * 0.8)
    df_train = df_copy.iloc[:split_idx].copy()
    df_test = df_copy.iloc[split_idx:].copy()
    h = len(df_test)

    # Preparar datos para StatsForecast
    df_train_model = df_train.rename(
        columns={'itemcode': 'unique_id', 'cantidad': 'y'}
    )[['unique_id', 'ds', 'y']]
    
    # Convertir el conjunto de entrenamiento en un array para la función mase
    y_train_array = df_train_model['y'].values
    
    # Grid de parámetros
    alphas_d = np.linspace(0.01, 0.5, 10)
    alphas_p = np.linspace(0.01, 0.5, 10)
    
    best_mase = float('inf')
    best_params = {}
    
    # Búsqueda de hiperparámetros
    for alpha_d in alphas_d:
        for alpha_p in alphas_p:
            model = TSB(alpha_d=alpha_d, alpha_p=alpha_p)
            sf = StatsForecast(models=[model], freq=freq)
            
            cv_df = sf.cross_validation(df=df_train_model, h=h, n_windows=1)
            
            # Calcular MASE en lugar de MAPE
            current_mase = mase(cv_df['y'].values, cv_df['TSB'].values, y_train_array)

            if current_mase < best_mase:
                best_mase = current_mase
                best_params = {'alpha_d': alpha_d, 'alpha_p': alpha_p}

    # Entrenar modelo final con mejores parámetros
    final_model = TSB(alpha_d=best_params['alpha_d'], alpha_p=best_params['alpha_p'])
    final_sf = StatsForecast(models=[final_model], freq=freq)
    final_sf.fit(df=df_train_model)

    # Predecir sobre h pasos
    forecast = final_sf.predict(h=h)

    # Agregar predicciones al test y los resultados
    df_test = df_test.copy()
    df_test['Prediccion'] = forecast['TSB'].values
    
    # Agregar las mejores métricas y parámetros para referencia
    df_test['MASE'] = best_mase

    return df_test

In [7]:
# Asegúrate de tener tu DataFrame llamado df_sku con columnas:
# ['itemcode', 'ds', 'cantidad']

resultado = train_tsb_statsforecast_tuned(df_sku)
print(resultado)


   itemcode         ds  cantidad   mesaño  Prediccion      MASE
16  SKU_001 2024-04-28         2  2024-17    0.199062  0.711773
17  SKU_001 2024-05-05         0  2024-18    0.199062  0.711773
18  SKU_001 2024-05-12         0  2024-19    0.199062  0.711773
19  SKU_001 2024-05-19         6  2024-20    0.199062  0.711773


In [8]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import ParameterGrid

# Implementación de MASE (Error Porcentual Absoluto Escalonado)
def mase(y_true, y_pred, y_train):
    """
    Calcula el Error Porcentual Absoluto Escalonado (MASE).
    
    Args:
        y_true (np.array): Valores reales.
        y_pred (np.array): Valores pronosticados.
        y_train (np.array): Valores del conjunto de entrenamiento.
        
    Returns:
        float: El valor del MASE.
    """
    # Numerador: Error absoluto de las predicciones
    abs_errors = np.abs(y_true - y_pred)
    
    # Denominador: Error absoluto del pronóstico ingenuo (naive forecast)
    denominator = np.mean(np.abs(y_train[1:] - y_train[:-1]))
    
    # Evita la división por cero
    if denominator == 0:
        return np.inf
    
    return np.mean(abs_errors) / denominator

def train_prophet_tuned(df, freq='W'):
    """
    Entrena el modelo Prophet con ajuste de parámetros (changepoint_prior_scale y seasonality_prior_scale)
    usando validación cruzada, optimizando el MASE.
    
    Args:
        df (pd.DataFrame): DataFrame de entrada con columnas ['itemcode', 'ds', 'cantidad'].
        freq (str): Frecuencia de la serie temporal. Ejemplo: 'D', 'W', 'M'.
        
    Returns:
        pd.DataFrame: El DataFrame de prueba con la columna 'Prediccion',
                      'Mejor_MASE' y 'Mejores_Parametros'.
    """
    # Asegurar formato correcto para Prophet
    df_prophet = df.rename(columns={'ds': 'ds', 'cantidad': 'y'})
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

    # Separar train/test
    split_idx = int(len(df_prophet) * 0.8)
    df_train = df_prophet.iloc[:split_idx].copy()
    df_test = df_prophet.iloc[split_idx:].copy()
    h = len(df_test)

    # Convertir el conjunto de entrenamiento en un array para el cálculo del MASE
    y_train_array = df_train['y'].values

    # Grid de parámetros para Prophet
    param_grid = {
        'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.2],
        'seasonality_prior_scale': [0.1, 1.0, 5.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative']
    }
    grid = ParameterGrid(param_grid)
    
    best_mase = float('inf')
    best_params = None

    # Búsqueda de hiperparámetros usando validación cruzada de Prophet
    for params in grid:
        m = Prophet(**params)
        m.fit(df_train)
        
        # Realizar validación cruzada
        # initial: Periodo inicial para la ventana de entrenamiento
        # period: Periodo entre las ventanas de corte
        # horizon: Horizonte de pronóstico
        cv_df = cross_validation(m, initial=f'{len(df_train)//2}{freq}', period=f'{h}{freq}', horizon=f'{h}{freq}')
        
        # Calcular MASE para esta combinación de parámetros
        current_mase = mase(cv_df['y'].values, cv_df['yhat'].values, y_train_array)
        
        if current_mase < best_mase:
            best_mase = current_mase
            best_params = params

    # Entrenar el modelo final con los mejores parámetros
    final_model = Prophet(**best_params)
    final_model.fit(df_train)

    # Predecir sobre h pasos en el futuro
    future = final_model.make_future_dataframe(periods=h, freq=freq, include_history=False)
    forecast = final_model.predict(future)

    # Agregar predicciones al DataFrame de test
    df_test['Prediccion'] = forecast['yhat'].values

    # Agregar las mejores métricas y parámetros para referencia
    df_test['Mejor_MASE'] = best_mase

    return df_test

Importing plotly failed. Interactive plots will not work.


In [9]:
# Asegúrate de tener tu DataFrame llamado df_sku con columnas:
# ['itemcode', 'ds', 'cantidad']

resultado = train_prophet_tuned(df_sku)
print(resultado)


22:07:46 - cmdstanpy - INFO - Chain [1] start processing
22:07:47 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/1 [00:00<?, ?it/s]22:07:47 - cmdstanpy - INFO - Chain [1] start processing
22:07:47 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 1/1 [00:00<00:00,  4.31it/s]
22:07:47 - cmdstanpy - INFO - Chain [1] start processing
22:07:47 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/1 [00:00<?, ?it/s]22:07:47 - cmdstanpy - INFO - Chain [1] start processing
22:07:47 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 1/1 [00:00<00:00,  4.65it/s]
22:07:47 - cmdstanpy - INFO - Chain [1] start processing
22:07:47 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/1 [00:00<?, ?it/s]22:07:48 - cmdstanpy - INFO - Chain [1] start processing
22:07:48 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 1/1 [00:00<00:00,  5.02it/s]
22:07:48 - cmdstanpy - INFO - Chain [1] start processing
22:07:48 - cmdst

   itemcode         ds  y   mesaño  Prediccion  Mejor_MASE
16  SKU_001 2024-04-28  2  2024-17    1.406585    0.818306
17  SKU_001 2024-05-05  0  2024-18    1.432114    0.818306
18  SKU_001 2024-05-12  0  2024-19    1.457642    0.818306
19  SKU_001 2024-05-19  6  2024-20    1.483170    0.818306
